### Movie Recommendation Using K Nearest Neighbors

#### Imports

In [212]:
import pandas as pd
import numpy as np
import ast
from sklearn.feature_extraction.text import CountVectorizer

In [263]:
from sklearn.metrics.pairwise import cosine_similarity

In [264]:
from sklearn.neighbors import NearestNeighbors

In [312]:
import pickle

#### Datasets

In [215]:
credits = pd.read_csv('tmdb_movie_credits.csv')

In [216]:
movies = pd.read_csv('tmdb_movie_dataset.csv')

#### Data Cleaning and Preprocessing

In [217]:
movies

,budget,genres,homepage,tmdbId,keywords,original_language,original_title,overview,popularity,production_companies,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,ratingId
0,4000000,"[{""id"": 80, ""name"": ""Crime""}, {""id"": 35, ""name...",NaN,5,"[{""id"": 612, ""name"": ""hotel""}, {""id"": 613, ""na...",en,Four Rooms,It's Ted the Bellhop's first night on the job....,22.876230,"[{""name"": ""Miramax Films"", ""id"": 14}, {""name"":...",...,1995-12-09,4300000,98.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,6.5,530,18
1,11000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 28, ""...",http://www.starwars.com/films/star-wars-episod...,11,"[{""id"": 803, ""name"": ""android""}, {""id"": 4270, ...",en,Star Wars,Princess Leia is captured and held hostage by ...,126.393695,"[{""name"": ""Lucasfilm"", ""id"": 1}, {""name"": ""Twe...",...,1977-05-25,775398007,121.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"A long time ago in a galaxy far, far away...",Star Wars,8.1,6624,260
2,94000000,"[{""id"": 16, ""name"": ""Animation""}, {""id"": 10751...",http://movies.disney.com/finding-nemo,12,"[{""id"": 494, ""name"": ""father son relationship""...",en,Finding Nemo,"Nemo, an adventurous young clownfish, is unexp...",85.688789,"[{""name"": ""Pixar Animation Studios"", ""id"": 3}]",...,2003-05-30,940335536,100.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"There are 3.7 trillion fish in the ocean, they...",Finding Nemo,7.6,6122,6377
3,55000000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",NaN,13,"[{""id"": 422, ""name"": ""vietnam veteran""}, {""id""...",en,Forrest Gump,A man with a low IQ has accomplished great thi...,138.133331,"[{""name"": ""Paramount Pictures"", ""id"": 4}]",...,1994-07-06,677945399,142.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"The world will never be the same, once you've ...",Forrest Gump,8.2,7927,356
4,15000000,"[{""id"": 18, ""name"": ""Drama""}]",http://www.dreamworks.com/ab/,14,"[{""id"": 255, ""name"": ""male nudity""}, {""id"": 29...",en,American Beauty,"Lester Burnham, a depressed suburban father in...",80.878605,"[{""name"": ""DreamWorks SKG"", ""id"": 27}, {""name""...",...,1999-09-15,356296601,122.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Look closer.,American Beauty,7.9,3313,2858
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4597,2000000,"[{""id"": 18, ""name"": ""Drama""}]",http://www.thedoglovermovie.com/,394047,"[{""id"": 1568, ""name"": ""undercover""}, {""id"": 15...",en,The Dog Lover,Sara Gold is a young girl on a quest to save m...,1.193219,"[{""name"": ""ESX Productions"", ""id"": 37081}, {""n...",...,2016-07-08,0,101.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,She ended up on the wrong side of right.,The Dog Lover,5.3,6,160860
4598,0,"[{""id"": 27, ""name"": ""Horror""}]",NaN,396152,[],en,Restoration,"During home renovations, a young couple releas...",2.525569,[],...,2016-05-03,0,90.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,Restoration,5.3,11,158390
4599,0,"[{""id"": 27, ""name"": ""Horror""}]",NaN,402515,"[{""id"": 321, ""name"": ""terror""}, {""id"": 8087, ""...",pt,Solitude,After finding an old storage locker filled wit...,0.018716,"[{""name"": ""Gravitas Ventures"", ""id"": 44632}]",...,2014-10-21,0,89.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,American Beast,0.0,0,171713
4600,0,"[{""id"": 36, ""name"": ""History""}, {""id"": 18, ""na...",NaN,407887,"[{""id"": 10586, ""name"": ""korea""}, {""id"": 229851...",ko,인천상륙작전,A squad of soldiers fight in the Korean War's ...,6.116436,"[{""name"": ""Taewon Entertainment"", ""id"": 3965}]",...,2016-07-27,0,110.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,The Odds Were 5000 to 1 … One was all They Nee...,Operat

In [218]:
print(movies.shape)
print(movies['original_language'].value_counts())

(4602, 21)
original_language
en    4328
fr      66
es      31
de      26
zh      24
ja      15
hi      15
it      12
cn      12
ko      10
ru      10
pt       8
da       7
sv       5
fa       4
nl       4
he       3
th       3
id       2
cs       2
ar       2
af       1
ro       1
no       1
vi       1
ps       1
el       1
hu       1
nb       1
xx       1
pl       1
is       1
te       1
ky       1
Name: count, dtype: int64


In [219]:
#List of columns to add from movies:
#title, tmdbid, genres, overview, keywords

In [220]:
movies = movies[['tmdbId','title','genres','overview','keywords']]

In [221]:
movies

,tmdbId,title,genres,overview,keywords
0,5,Four Rooms,"[{""id"": 80, ""name"": ""Crime""}, {""id"": 35, ""name...",It's Ted the Bellhop's first night on the job....,"[{""id"": 612, ""name"": ""hotel""}, {""id"": 613, ""na..."
1,11,Star Wars,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 28, ""...",Princess Leia is captured and held hostage by ...,"[{""id"": 803, ""name"": ""android""}, {""id"": 4270, ..."
2,12,Finding Nemo,"[{""id"": 16, ""name"": ""Animation""}, {""id"": 10751...","Nemo, an adventurous young clownfish, is unexp...","[{""id"": 494, ""name"": ""father son relationship""..."
3,13,Forrest Gump,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",A man with a low IQ has accomplished great thi...,"[{""id"": 422, ""name"": ""vietnam veteran""}, {""id""..."
4,14,American Beauty,"[{""id"": 18, ""name"": ""Drama""}]","Lester Burnham, a depressed suburban father in...","[{""id"": 255, ""name"": ""male nudity""}, {""id"": 29..."
...,...,...,...,...,...
4597,394047,The Dog Lover,"[{""id"": 18, ""name"": ""Drama""}]",Sara Gold is a young girl on a quest to save m...,"[{""id"": 1568, ""name"": ""undercover""}, {""id"": 15..."
4598,396152,Restoration,"[{""id"": 27, ""name"": ""Horror""}]","During home renovations, a young couple releas...",[]
4599,402515,American Beast,"[{""id"": 27, ""name"": ""Horror""}]",After finding an old storage locker filled wit...,"[{""id"": 321, ""name"": ""terror""}, {""id"": 8087, ""..."
4600,407887,Operation Chromite,"[{""id"": 36, ""name"": ""History""}, {""id"": 18, ""na...",A squad of soldiers fight in the Korean War's ...,"[{""id"": 10586, ""name"": ""korea""}, {""id"": 229851..."


In [222]:
print(movies['title'].isnull().sum())
print(movies['tmdbId'].isnull().sum())
print(movies['genres'].isnull().sum())
print(movies['overview'].isnull().sum())
print(movies['keywords'].isnull().sum())

0
0
0
1
0


In [223]:
credits

,tmdbId,title,cast,crew
0,5,Four Rooms,"[{""cast_id"": 42, ""character"": ""Ted the Bellhop...","[{""credit_id"": ""52fe420dc3a36847f800012d"", ""de..."
1,11,Star Wars,"[{""cast_id"": 3, ""character"": ""Luke Skywalker"",...","[{""credit_id"": ""52fe420dc3a36847f8000437"", ""de..."
2,12,Finding Nemo,"[{""cast_id"": 8, ""character"": ""Marlin (voice)"",...","[{""credit_id"": ""52fe420ec3a36847f80006b1"", ""de..."
3,13,Forrest Gump,"[{""cast_id"": 7, ""character"": ""Forrest Gump"", ""...","[{""credit_id"": ""52fe420ec3a36847f800076b"", ""de..."
4,14,American Beauty,"[{""cast_id"": 6, ""character"": ""Lester Burnham"",...","[{""credit_id"": ""52fe420ec3a36847f8000809"", ""de..."
...,...,...,...,...
4597,394047,The Dog Lover,"[{""cast_id"": 4, ""character"": ""Sara Gold"", ""cre...","[{""credit_id"": ""58ec10d592514152ac04483f"", ""de..."
4598,396152,Restoration,"[{""cast_id"": 3, ""character"": ""Rebecca Jordan"",...","[{""credit_id"": ""572bd470c3a368013d0002b1"", ""de..."
4599,402515,American Beast,"[{""cast_id"": 0, ""character"": ""James Erikson"", ...","[{""credit_id"": ""57edab3b9251416cd600077f"", ""de..."
4600,407887,Operation Chromite,"[{""cast_id"": 1, ""character"": ""Jang, Hak-soo (S...","[{""credit_id"": ""5843fb6fc3a3684813007fb4"", ""de..."


In [224]:
#List of columns to add from credits:
#title, tmdbid, cast, crew

In [225]:
print(credits['title'].isnull().sum())
print(credits['tmdbId'].isnull().sum())
print(credits['cast'].isnull().sum())
print(credits['crew'].isnull().sum())

0
0
0
0


In [226]:
movies_data = pd.merge(movies, credits, on=['title', 'tmdbId'], how='inner') 

In [227]:
movies_data

,tmdbId,title,genres,overview,keywords,cast,crew
0,5,Four Rooms,"[{""id"": 80, ""name"": ""Crime""}, {""id"": 35, ""name...",It's Ted the Bellhop's first night on the job....,"[{""id"": 612, ""name"": ""hotel""}, {""id"": 613, ""na...","[{""cast_id"": 42, ""character"": ""Ted the Bellhop...","[{""credit_id"": ""52fe420dc3a36847f800012d"", ""de..."
1,11,Star Wars,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 28, ""...",Princess Leia is captured and held hostage by ...,"[{""id"": 803, ""name"": ""android""}, {""id"": 4270, ...","[{""cast_id"": 3, ""character"": ""Luke Skywalker"",...","[{""credit_id"": ""52fe420dc3a36847f8000437"", ""de..."
2,12,Finding Nemo,"[{""id"": 16, ""name"": ""Animation""}, {""id"": 10751...","Nemo, an adventurous young clownfish, is unexp...","[{""id"": 494, ""name"": ""father son relationship""...","[{""cast_id"": 8, ""character"": ""Marlin (voice)"",...","[{""credit_id"": ""52fe420ec3a36847f80006b1"", ""de..."
3,13,Forrest Gump,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",A man with a low IQ has accomplished great thi...,"[{""id"": 422, ""name"": ""vietnam veteran""}, {""id""...","[{""cast_id"": 7, ""character"": ""Forrest Gump"", ""...","[{""credit_id"": ""52fe420ec3a36847f800076b"", ""de..."
4,14,American Beauty,"[{""id"": 18, ""name"": ""Drama""}]","Lester Burnham, a depressed suburban father in...","[{""id"": 255, ""name"": ""male nudity""}, {""id"": 29...","[{""cast_id"": 6, ""character"": ""Lester Burnham"",...","[{""credit_id"": ""52fe420ec3a36847f8000809"", ""de..."
...,...,...,...,...,...,...,...
4605,394047,The Dog Lover,"[{""id"": 18, ""name"": ""Drama""}]",Sara Gold is a young girl on a quest to save m...,"[{""id"": 1568, ""name"": ""undercover""}, {""id"": 15...","[{""cast_id"": 4, ""character"": ""Sara Gold"", ""cre...","[{""credit_id"": ""58ec10d592514152ac04483f"", ""de..."
4606,396152,Restoration,"[{""id"": 27, ""name"": ""Horror""}]","During home renovations, a young couple releas...",[],"[{""cast_id"": 3, ""character"": ""Rebecca Jordan"",...","[{""credit_id"": ""572bd470c3a368013d0002b1"", ""de..."
4607,402515,American Beast,"[{""id"": 27, ""name"": ""Horror""}]",After finding an old storage locker filled wit...,"[{""id"": 321, ""name"": ""terror""}, {""id"": 8087, ""...","[{""cast_id"": 0, ""character"": ""James Erikson"", ...","[{""credit_id"": ""57edab3b9251416cd600077f"", ""de..."
4608,407887,Operation Chromite,"[{""id"": 36, ""name"": ""History""}, {""id"": 18, ""na...",A squad of soldiers fight in the Korean War's ...,"[{""id"": 10586, ""name"": ""korea""}, {""id"": 229851...","[{""cast_id"": 1, ""character"": ""Jang, Hak-soo (S...","[{""credit_id"": ""5843fb6fc3a3684813007fb4"", ""de..."


In [228]:
print(movies_data['title'].isnull().sum())
print(movies_data['tmdbId'].isnull().sum())
print(movies_data['cast'].isnull().sum())
print(movies_data['crew'].isnull().sum())
print(movies_data['genres'].isnull().sum())
print(movies_data['overview'].isnull().sum())
print(movies_data['keywords'].isnull().sum())

0
0
0
0
0
1
0


In [229]:
movies_data.dropna(inplace = True)

In [230]:
movies_data.shape

(4609, 7)

In [231]:
movies_data.iloc[0]['genres']

'[{"id": 80, "name": "Crime"}, {"id": 35, "name": "Comedy"}]'

In [232]:
print("Genres = " + movies_data.iloc[0]['genres'])
print("Keywords = " + movies_data.iloc[0]['keywords'])
print("Cast = " + movies_data.iloc[0]['cast'])
print("Crew = " + movies_data.iloc[0]['crew'])
print("Overview = " + movies_data.iloc[0]['overview'])

Genres = [{"id": 80, "name": "Crime"}, {"id": 35, "name": "Comedy"}]
Keywords = [{"id": 612, "name": "hotel"}, {"id": 613, "name": "new year's eve"}, {"id": 616, "name": "witch"}, {"id": 622, "name": "bet"}, {"id": 922, "name": "hotel room"}, {"id": 2700, "name": "sperm"}, {"id": 12670, "name": "los angeles"}, {"id": 160488, "name": "hoodlum"}, {"id": 187056, "name": "woman director"}, {"id": 198129, "name": "episode film"}]
Cast = [{"cast_id": 42, "character": "Ted the Bellhop", "credit_id": "52fe420dc3a36847f80001b7", "gender": 2, "id": 3129, "name": "Tim Roth", "order": 0}, {"cast_id": 31, "character": "Man", "credit_id": "52fe420dc3a36847f800018b", "gender": 2, "id": 3131, "name": "Antonio Banderas", "order": 1}, {"cast_id": 29, "character": "Angela", "credit_id": "52fe420dc3a36847f8000183", "gender": 1, "id": 3130, "name": "Jennifer Beals", "order": 2}, {"cast_id": 25, "character": "Elspeth", "credit_id": "52fe420dc3a36847f8000173", "gender": 1, "id": 3125, "name": "Madonna", "ord

In [233]:
def transform_data(list_of_dict):
    result = list()
    for item in ast.literal_eval(list_of_dict):
        result.append(item['name'])
    return result

In [234]:
print(transform_data('[{"id": 80, "name": "Crime"}, {"id": 35, "name": "Comedy"}]'))

['Crime', 'Comedy']


In [235]:
movies_data['genres'] = movies_data['genres'].apply(transform_data)

In [236]:
movies_data['keywords'] = movies_data['keywords'].apply(transform_data)

In [237]:
def transform_data_top3(list_of_dict):
    result = list()
    for item in ast.literal_eval(list_of_dict):
        if len(result) < 3:
            result.append(item['name'])
        else:
            break
    return result

In [238]:
print(transform_data_top3('[{"cast_id": 42, "character": "Ted the Bellhop", "credit_id": "52fe420dc3a36847f80001b7", "gender": 2, "id": 3129, "name": "Tim Roth", "order": 0}, {"cast_id": 31, "character": "Man", "credit_id": "52fe420dc3a36847f800018b", "gender": 2, "id": 3131, "name": "Antonio Banderas", "order": 1}, {"cast_id": 29, "character": "Angela", "credit_id": "52fe420dc3a36847f8000183", "gender": 1, "id": 3130, "name": "Jennifer Beals", "order": 2}, {"cast_id": 25, "character": "Elspeth", "credit_id": "52fe420dc3a36847f8000173", "gender": 1, "id": 3125, "name": "Madonna", "order": 3}, {"cast_id": 41, "character": "Margaret", "credit_id": "52fe420dc3a36847f80001b3", "gender": 1, "id": 3141, "name": "Marisa Tomei", "order": 4}, {"cast_id": 43, "character": "Leo", "credit_id": "52fe420dc3a36847f80001bb", "gender": 2, "id": 62, "name": "Bruce Willis", "order": 5}, {"cast_id": 38, "character": "Chester Rush", "credit_id": "52fe420dc3a36847f80001a7", "gender": 2, "id": 138, "name": "Quentin Tarantino", "order": 6}, {"cast_id": 22, "character": "Jezebel", "credit_id": "52fe420dc3a36847f8000167", "gender": 1, "id": 3122, "name": "Sammi Davis", "order": 7}, {"cast_id": 23, "character": "Diana", "credit_id": "52fe420dc3a36847f800016b", "gender": 1, "id": 3123, "name": "Amanda de Cadenet", "order": 8}, {"cast_id": 24, "character": "Athena", "credit_id": "52fe420dc3a36847f800016f", "gender": 1, "id": 3124, "name": "Valeria Golino", "order": 9}, {"cast_id": 27, "character": "Raven", "credit_id": "52fe420dc3a36847f800017b", "gender": 1, "id": 3127, "name": "Lili Taylor", "order": 10}, {"cast_id": 39, "character": "Long Hair Yuppy Scum", "credit_id": "52fe420dc3a36847f80001ab", "gender": 2, "id": 2545, "name": "Lawrence Bender", "order": 11}, {"cast_id": 26, "character": "Eva", "credit_id": "52fe420dc3a36847f8000177", "gender": 1, "id": 3126, "name": "Ione Skye", "order": 12}, {"cast_id": 28, "character": "Kiva", "credit_id": "52fe420dc3a36847f800017f", "gender": 1, "id": 3128, "name": "Alicia Witt", "order": 13}, {"cast_id": 30, "character": "Siegfried", "credit_id": "52fe420dc3a36847f8000187", "gender": 2, "id": 2555, "name": "David Proval", "order": 14}, {"cast_id": 32, "character": "Sarah", "credit_id": "52fe420dc3a36847f800018f", "gender": 1, "id": 3132, "name": "Lana McKissack", "order": 15}, {"cast_id": 34, "character": "Wife", "credit_id": "52fe420dc3a36847f8000197", "gender": 1, "id": 3134, "name": "Tamlyn Tomita", "order": 16}, {"cast_id": 40, "character": "Betty", "credit_id": "52fe420dc3a36847f80001af", "gender": 1, "id": 3138, "name": "Kathy Griffin", "order": 17}, {"cast_id": 37, "character": "Norman", "credit_id": "52fe420dc3a36847f80001a3", "gender": 2, "id": 3137, "name": "Paul Calderon", "order": 18}, {"cast_id": 36, "character": "TV Dancing Girl", "credit_id": "52fe420dc3a36847f800019f", "gender": 1, "id": 3136, "name": "Salma Hayek", "order": 19}, {"cast_id": 33, "character": "Corpse", "credit_id": "52fe420dc3a36847f8000193", "gender": 1, "id": 3133, "name": "Patricia Vonne", "order": 20}, {"cast_id": 35, "character": "Juancho", "credit_id": "52fe420dc3a36847f800019b", "gender": 0, "id": 3135, "name": "Danny Verduzco", "order": 21}, {"cast_id": 44, "character": "Hooker", "credit_id": "52fe420dc3a36847f80001bf", "gender": 0, "id": 37336, "name": "Kimberly Blair", "order": 22}, {"cast_id": 45, "character": "Sam the Bellhop", "credit_id": "52fe420dc3a36847f80001c3", "gender": 2, "id": 3140, "name": "Marc Lawrence", "order": 23}]'))

['Tim Roth', 'Antonio Banderas', 'Jennifer Beals']


In [239]:
movies_data['cast'] = movies_data['cast'].apply(transform_data_top3)

In [240]:
def transform_crew(list_of_dict):
    result = []
    for item in ast.literal_eval(list_of_dict):
        if item['job'] == 'Director':
            result.append(item['name'])
            break
    return result

In [241]:
movies_data['crew'].apply(transform_crew)

0        [Allison Anders]
1          [George Lucas]
2        [Andrew Stanton]
3       [Robert Zemeckis]
4            [Sam Mendes]
              ...        
4605    [Alex Ranarivelo]
4606          [Zack Ward]
4607    [Livingston Oden]
4608        [John H. Lee]
4609       [Frank Lotito]
Name: crew, Length: 4609, dtype: object

In [242]:
movies_data['crew'] = movies_data['crew'].apply(transform_crew)

In [243]:
movies_data['overview'].apply(lambda word : word.split())

0       [It's, Ted, the, Bellhop's, first, night, on, ...
1       [Princess, Leia, is, captured, and, held, host...
2       [Nemo,, an, adventurous, young, clownfish,, is...
3       [A, man, with, a, low, IQ, has, accomplished, ...
4       [Lester, Burnham,, a, depressed, suburban, fat...
                              ...                        
4605    [Sara, Gold, is, a, young, girl, on, a, quest,...
4606    [During, home, renovations,, a, young, couple,...
4607    [After, finding, an, old, storage, locker, fil...
4608    [A, squad, of, soldiers, fight, in, the, Korea...
4609    [In, 1979,, an, Indian, family, moves, to, Ame...
Name: overview, Length: 4609, dtype: object

In [244]:
movies_data['overview'] = movies_data['overview'].apply(lambda sentence : sentence.split())

In [245]:
movies_data.head()

,tmdbId,title,genres,overview,keywords,cast,crew
0,5,Four Rooms,"[Crime, Comedy]","[It's, Ted, the, Bellhop's, first, night, on, ...","[hotel, new year's eve, witch, bet, hotel room...","[Tim Roth, Antonio Banderas, Jennifer Beals]",[Allison Anders]
1,11,Star Wars,"[Adventure, Action, Science Fiction]","[Princess, Leia, is, captured, and, held, host...","[android, galaxy, hermit, death star, lightsab...","[Mark Hamill, Harrison Ford, Carrie Fisher]",[George Lucas]
2,12,Finding Nemo,"[Animation, Family]","[Nemo,, an, adventurous, young, clownfish,, is...","[father son relationship, harbor, underwater, ...","[Albert Brooks, Ellen DeGeneres, Alexander Gould]",[Andrew Stanton]
3,13,Forrest Gump,"[Comedy, Drama, Romance]","[A, man, with, a, low, IQ, has, accomplished, ...","[vietnam veteran, hippie, mentally disabled, r...","[Tom Hanks, Robin Wright, Gary Sinise]",[Robert Zemeckis]
4,14,American Beauty,[Drama],"[Lester, Burnham,, a, depressed, suburban, fat...","[male nudity, female nudity, adultery, midlife...","[Kevin Spacey, Annette Bening, Thora Birch]",[Sam Mendes]


In [246]:
movies_data['crew'].apply(lambda director_list : [name.replace(" ", "") for name in director_list])

0        [AllisonAnders]
1          [GeorgeLucas]
2        [AndrewStanton]
3       [RobertZemeckis]
4            [SamMendes]
              ...       
4605    [AlexRanarivelo]
4606          [ZackWard]
4607    [LivingstonOden]
4608         [JohnH.Lee]
4609       [FrankLotito]
Name: crew, Length: 4609, dtype: object

In [247]:
movies_data['crew'] = movies_data['crew'].apply(lambda director_list : [name.replace(" ", "") for name in director_list])

In [248]:
movies_data['cast'] = movies_data['cast'].apply(lambda actors_list : [name.replace(" ", "") for name in actors_list])

In [249]:
movies_data['genres'] = movies_data['genres'].apply(lambda genres_list : [name.replace(" ", "") for name in genres_list])

In [250]:
movies_data['keywords'] = movies_data['keywords'].apply(lambda keywords_list : [name.replace(" ", "") for name in keywords_list])

In [251]:
movies_data.head()

,tmdbId,title,genres,overview,keywords,cast,crew
0,5,Four Rooms,"[Crime, Comedy]","[It's, Ted, the, Bellhop's, first, night, on, ...","[hotel, newyear'seve, witch, bet, hotelroom, s...","[TimRoth, AntonioBanderas, JenniferBeals]",[AllisonAnders]
1,11,Star Wars,"[Adventure, Action, ScienceFiction]","[Princess, Leia, is, captured, and, held, host...","[android, galaxy, hermit, deathstar, lightsabe...","[MarkHamill, HarrisonFord, CarrieFisher]",[GeorgeLucas]
2,12,Finding Nemo,"[Animation, Family]","[Nemo,, an, adventurous, young, clownfish,, is...","[fathersonrelationship, harbor, underwater, fi...","[AlbertBrooks, EllenDeGeneres, AlexanderGould]",[AndrewStanton]
3,13,Forrest Gump,"[Comedy, Drama, Romance]","[A, man, with, a, low, IQ, has, accomplished, ...","[vietnamveteran, hippie, mentallydisabled, run...","[TomHanks, RobinWright, GarySinise]",[RobertZemeckis]
4,14,American Beauty,[Drama],"[Lester, Burnham,, a, depressed, suburban, fat...","[malenudity, femalenudity, adultery, midlifecr...","[KevinSpacey, AnnetteBening, ThoraBirch]",[SamMendes]


In [252]:
movies_data['words'] = movies_data['genres']  + movies_data['cast'] + movies_data['crew'] + movies_data['keywords'] + movies_data['overview']

In [253]:
movies_data = movies_data[['tmdbId', 'title', 'words']]

movies_data.head()

In [255]:
movies_data['words'] = movies_data['words'].apply(lambda word_list : " ".join(word_list))

C:\Users\palri\AppData\Local\Temp\ipykernel_14956\3097958515.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_data['words'] = movies_data['words'].apply(lambda word_list : " ".join(word_list))


In [256]:
movies_data['words'] = movies_data['words'].apply(lambda word_str : word_str.lower())

C:\Users\palri\AppData\Local\Temp\ipykernel_14956\2298117169.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_data['words'] = movies_data['words'].apply(lambda word_str : word_str.lower())


In [257]:
movies_data.head()

,tmdbId,title,words
0,5,Four Rooms,crime comedy timroth antoniobanderas jenniferb...
1,11,Star Wars,adventure action sciencefiction markhamill har...
2,12,Finding Nemo,animation family albertbrooks ellendegeneres a...
3,13,Forrest Gump,comedy drama romance tomhanks robinwright gary...
4,14,American Beauty,drama kevinspacey annettebening thorabirch sam...


#### Vectorization

In [283]:
cv = CountVectorizer(max_features=6000, stop_words='english')
movie_vectors = cv.fit_transform(movies_data['words']).toarray()
print(movie_vectors)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]]


#### K Nearest Neighbors

In [287]:
# Fit KNN model
knn = NearestNeighbors(n_neighbors=6, metric='cosine')  
knn.fit(movie_vectors)

NearestNeighbors(metric='cosine', n_neighbors=6)

In [295]:
# Get distances and indices for all movies
distances, indices = knn.kneighbors(movie_vectors)

# Sort each movie's neighbors based on distance
recommendations_list_via_index = np.array([indices[i][np.argsort(distances[i])] for i in range(len(movie_vectors))])

In [310]:
print(recommendations_list_via_index)
print(indices)

[[   0  802 3711 4383 4096  778]
 [   1  601  602  605  603 1120]
 [   2 2378 1727 2191 3689 3665]
 ...
 [4606 4585 4412 4356 1872 4011]
 [4607 3955 1735 4415 3691 2489]
 [4608 4299 1872 4486 4096 2394]]
[[   0  802 3711 4383 4096  778]
 [   1  601  602  605  603 1120]
 [   2 2378 1727 2191 3689 3665]
 ...
 [4606 4585 4412 4356 1872 4011]
 [4607 3955 1735 4415 3691 2489]
 [4608 4299 1872 4486 4096 2394]]


In [309]:
print(recommendations_list_via_index.shape)
print(indices.shape)

(4609, 6)
(4609, 6)


In [300]:
def recommend(title):
    movie_index = movies_data[movies_data['title'] == title].index[0]
    list_of_recommendations = recommendations_list_via_index[movie_index]
    for index in list_of_recommendations[1:6]:
        print(movies_data.iloc[index]['title'])
        print()

In [301]:
recommend('Star Wars')

The Empire Strikes Back

Return of the Jedi

Star Wars: Episode III - Revenge of the Sith

Star Wars: Episode I - The Phantom Menace

Shanghai Noon



In [311]:
recommend('Batman Begins')

The Dark Knight

The Dark Knight Rises

Batman

Batman & Robin

Batman v Superman: Dawn of Justice



In [311]:
recommend('Batman Begins')

The Dark Knight

The Dark Knight Rises

Batman

Batman & Robin

Batman v Superman: Dawn of Justice



#### Exports for WebApp

In [314]:
pickle.dump(movies_data, open('movies_data.pkl', 'wb'))

In [316]:
pickle.dump(recommendations_list_via_index, open('recommendations.pkl', 'wb'))